In [1]:
# import libraries
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd# specify the url

In [2]:
urlpage = 'https://stats.nba.com/players/list/?Historic=Y' 
print(urlpage)
# run firefox webdriver from executable path of your choice
driver = webdriver.Firefox()

https://stats.nba.com/players/list/?Historic=Y


In [3]:
# get web page
driver.get(urlpage)
# execute script to scroll down the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
# sleep for 30s
time.sleep(30)
# driver.quit()

In [4]:
# find elements by xpath# at time of publication, Nov 2018:
# results = driver.find_elements_by_xpath("//*[@id='componentsContainer']//*[contains(@id,'listingsContainer')]//*[@class='product active']//*[@class='title productTitle']")# updated Nov 2019:
results = driver.find_elements_by_xpath("//*[@class='stats-player-list players-list']//*[@class='row collapse players-list__section']//*[@class='players-list__names']//*[@class='players-list__name']")
print('Number of results', len(results))

Number of results 4503


In [5]:
product_name = results.text

AttributeError: 'list' object has no attribute 'text'

In [ ]:
product_name

In [ ]:
player = result.find_element_by_tag_name('a')

In [ ]:
player

In [ ]:
link

In [6]:
# create empty array to store data
data = []
# loop over results
for result in results:
    product_name = result.text
    player = result.find_element_by_tag_name('a')
    link = player.get_attribute("href")
    # append dict to array
    data.append({"player" : product_name, "link" : link})

In [38]:
product_name

'Zunic, Matt'

In [39]:
# close driver 
driver.quit()

In [9]:
# save to pandas dataframe
df = pd.DataFrame(data)
df

,player,link
0,"Abdelnaby, Alaa",https://stats.nba.com/player/76001/
1,"Abdul-Aziz, Zaid",https://stats.nba.com/player/76002/
2,"Abdul-Jabbar, Kareem",https://stats.nba.com/player/76003/
3,"Abdul-Rauf, Mahmoud",https://stats.nba.com/player/51/
4,"Abdul-Wahad, Tariq",https://stats.nba.com/player/1505/
...,...,...
4498,"Zizic, Ante",https://stats.nba.com/player/1627790/
4499,"Zoet, Jim",https://stats.nba.com/player/78647/
4500,"Zopf, Bill",https://stats.nba.com/player/78648/
4501,"Zubac, Ivica",https://stats.nba.com/player/1627826/


In [11]:
df.to_csv('stats_nba_links.csv')

In [ ]:
team = clean_schedule[['away','home']].copy()
team.drop(columns=['home'],inplace=True)
team.drop_duplicates(inplace=True)
team.sort_values('away', ascending=True).reset_index()

In [32]:
df['stats_id'] = df['link']

In [35]:
for x in df['stats_id']:
    df['stats_id'] = df['link'].str[29:-1]

In [25]:
len('https://stats.nba.com/player/')

29

In [36]:
df

,player,link,stats_id
0,"Abdelnaby, Alaa",https://stats.nba.com/player/76001/,76001
1,"Abdul-Aziz, Zaid",https://stats.nba.com/player/76002/,76002
2,"Abdul-Jabbar, Kareem",https://stats.nba.com/player/76003/,76003
3,"Abdul-Rauf, Mahmoud",https://stats.nba.com/player/51/,51
4,"Abdul-Wahad, Tariq",https://stats.nba.com/player/1505/,1505
...,...,...,...
4498,"Zizic, Ante",https://stats.nba.com/player/1627790/,1627790
4499,"Zoet, Jim",https://stats.nba.com/player/78647/,78647
4500,"Zopf, Bill",https://stats.nba.com/player/78648/,78648
4501,"Zubac, Ivica",https://stats.nba.com/player/1627826/,1627826


In [ ]:
urlpage = 'https://stats.nba.com/player/203507/usage/?Season=2015-16&SeasonType=Regular%20Season' 
print(urlpage)
# run firefox webdriver from executable path of your choice
driver = webdriver.Firefox()

In [ ]:
# get web page
driver.get(urlpage)
# execute script to scroll down the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
# sleep for 30s
time.sleep(30)
# driver.quit()

In [ ]:
df